In [1]:
import optuna

import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm

# Iris Dataset

In [2]:
def objective(trial):
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target

    classifier_name = trial.suggest_categorical("classifier", ["SVC", "RandomForest"])
    if classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
        classifier_obj = sklearn.svm.SVC(C = svc_c, gamma="auto")
    else:
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        classifier_obj = sklearn.ensemble.RandomForestClassifier(
            max_depth=rf_max_depth,
            n_estimators=10
        )

    score = sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy

In [6]:
study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///db.sqlite3",
    study_name="iris classifier"

)
study.optimize(objective, n_trials=100)

[I 2024-10-20 22:16:08,466] A new study created in RDB with name: iris classifier
[I 2024-10-20 22:16:08,627] Trial 0 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 0.0027111877194324198}. Best is trial 0 with value: 0.32.
[I 2024-10-20 22:16:08,768] Trial 1 finished with value: 0.9666666666666667 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 4}. Best is trial 1 with value: 0.9666666666666667.
[I 2024-10-20 22:16:08,901] Trial 2 finished with value: 0.9733333333333333 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 3}. Best is trial 2 with value: 0.9733333333333333.
[I 2024-10-20 22:16:09,011] Trial 3 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 8.792219697713163e-08}. Best is trial 2 with value: 0.9733333333333333.
[I 2024-10-20 22:16:09,123] Trial 4 finished with value: 0.96 and parameters: {'classifier': 'SVC', 'svc_c': 2934167.478337526}. Best is trial 2 with value: 0.9733333333333333.
[I 2024-10-

In [7]:
print(study.best_trial)

FrozenTrial(number=2, state=1, values=[0.9733333333333333], datetime_start=datetime.datetime(2024, 10, 20, 22, 16, 8, 774060), datetime_complete=datetime.datetime(2024, 10, 20, 22, 16, 8, 877223), params={'classifier': 'RandomForest', 'rf_max_depth': 3}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'classifier': CategoricalDistribution(choices=('SVC', 'RandomForest')), 'rf_max_depth': IntDistribution(high=32, log=True, low=2, step=1)}, trial_id=3, value=None)
